In [0]:
import cv2
import tensorflow
import keras
from keras.models import load_model
from scipy.spatial import distance as dist
import numpy as np
import matplotlib as plt
import os
import glob
from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
import glob

Using TensorFlow backend.


In [0]:
def segment_img(img,stride,npw,nph):
#     img = cv2.imread(img)
    height,width, color = img.shape
    test = 0
    # print(height,width,color)
    orig = img
    num_strides_height = height/stride
    num_strides_width = width/stride
    rows = int(num_strides_height)
    columns = int(num_strides_width)
    # print(rows,columns)
    for r in range(rows):
        for c in range(columns):
            x1 = (c)*stride
            y1 = (r)*stride
            x2 = x1+npw
            y2 = y1+nph
            if test == 0:
                in_img = img[y1:y2,x1:x2]
                if(len(in_img.shape)<3):
#                     in_img = in_img.reshape(1,227,227,1)
                    in_img = in_img/255
                else:
                    in_img = in_img.astype('uint8')
#                     in_img = cv2.cvtColor(in_img,cv2.COLOR_RGB2GRAY)
#                     in_img = in_img.reshape(1,227,227,1)
                    in_img = in_img/255
#The below line is used to predict
                pred = model.predict(in_img[np.newaxis,:,:,:])
                pred_class = np.argmax(pred)
#                 result = model.predict(in_img, batch_size=None, verbose=0, steps=None)
#                 re = result[0]
                if pred_class == 1:
                    print('cracked')
#                     print('confidence: ',re[0]*100,'%')
                    orig = cv2.rectangle(orig,(x1,y1),(x2,y2),(0,0,255),2)
                else:
                    jkl=0
                    # print('no crack')
            if ((width-x2 >= npw) or (width==npw)):
                test = 0
            else:
                break
        if ((height-y2) >= nph or (height==nph)):
            test=0
        else:
            break
    return(orig)


In [0]:
model = load_model('D://Crack_Detection/model.h5')
cap = cv2.VideoCapture(0)
stride = 50 #pixels to stride frwd or down
#Shape of neural net dectectable image width x height
neural_pixels_width = 224
neural_pixels_height = 224
npw = neural_pixels_width
nph = neural_pixels_height

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    results = segment_img(frame,stride,npw,nph)
    # Display the resulting frame
    cv2.imshow('frame',results)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
ca  p.release()
cv2.destroyAllWindows()

cracked
confidence:  124.42739009857178 %
cracked
confidence:  131.93711042404175 %
cracked
confidence:  126.25830173492432 %
cracked
confidence:  173.5337734222412 %
cracked
confidence:  123.12726974487305 %
cracked
confidence:  115.95972776412964 %
cracked
confidence:  114.29556608200073 %
cracked
confidence:  97.62598872184753 %
cracked
confidence:  109.77392196655273 %
cracked
confidence:  173.39799404144287 %
cracked
confidence:  131.17008209228516 %
cracked
confidence:  141.53125286102295 %
cracked
confidence:  122.18598127365112 %
cracked
confidence:  118.62235069274902 %
cracked
confidence:  115.21761417388916 %
cracked
confidence:  101.87070369720459 %
cracked
confidence:  81.68690204620361 %
cracked
confidence:  109.8020076751709 %
cracked
confidence:  173.2909083366394 %
cracked
confidence:  131.49142265319824 %
cracked
confidence:  141.9503927230835 %
cracked
confidence:  122.11098670959473 %
cracked
confidence:  118.58705282211304 %
cracked
confidence:  115.34334421157837 

cracked
confidence:  110.4062557220459 %
cracked
confidence:  171.3024616241455 %
cracked
confidence:  133.83164405822754 %
cracked
confidence:  144.2350149154663 %
cracked
confidence:  122.05228805541992 %
cracked
confidence:  118.55978965759277 %
cracked
confidence:  115.60255289077759 %
cracked
confidence:  101.55231952667236 %
cracked
confidence:  82.0074737071991 %
cracked
confidence:  110.39841175079346 %
cracked
confidence:  171.38588428497314 %
cracked
confidence:  133.80393981933594 %
cracked
confidence:  144.73875761032104 %
cracked
confidence:  122.24271297454834 %
cracked
confidence:  118.66731643676758 %
cracked
confidence:  115.79862833023071 %
cracked
confidence:  101.38146877288818 %
cracked
confidence:  81.7750632762909 %
cracked
confidence:  110.11950969696045 %
cracked
confidence:  171.22024297714233 %
cracked
confidence:  134.21777486801147 %
cracked
confidence:  144.73702907562256 %
cracked
confidence:  122.4139928817749 %
cracked
confidence:  118.73205900192261 %


cracked
confidence:  122.38826751708984 %
cracked
confidence:  118.8740611076355 %
cracked
confidence:  114.4162654876709 %
cracked
confidence:  99.2621660232544 %
cracked
confidence:  86.43524646759033 %
cracked
confidence:  109.53980684280396 %
cracked
confidence:  160.8129620552063 %
cracked
confidence:  142.0189619064331 %
cracked
confidence:  138.8260006904602 %
cracked
confidence:  122.10633754730225 %
cracked
confidence:  118.70337724685669 %
cracked
confidence:  114.29542303085327 %
cracked
confidence:  99.41381216049194 %
cracked
confidence:  87.02614903450012 %
cracked
confidence:  109.36717987060547 %
cracked
confidence:  160.6884479522705 %
cracked
confidence:  142.00358390808105 %
cracked
confidence:  138.108491897583 %
cracked
confidence:  122.20147848129272 %
cracked
confidence:  118.70253086090088 %
cracked
confidence:  114.1440749168396 %
cracked
confidence:  99.54584240913391 %
cracked
confidence:  87.1607780456543 %
cracked
confidence:  109.30240154266357 %
cracked
c

cracked
confidence:  87.63483166694641 %
cracked
confidence:  110.30054092407227 %
cracked
confidence:  159.74962711334229 %
cracked
confidence:  144.8625087738037 %
cracked
confidence:  137.83392906188965 %
cracked
confidence:  122.45068550109863 %
cracked
confidence:  118.9551830291748 %
cracked
confidence:  115.60657024383545 %
cracked
confidence:  100.13387203216553 %
cracked
confidence:  87.39914298057556 %
cracked
confidence:  110.3258728981018 %
cracked
confidence:  159.64679718017578 %
cracked
confidence:  144.87088918685913 %
cracked
confidence:  138.10921907424927 %
cracked
confidence:  122.29324579238892 %
cracked
confidence:  118.86656284332275 %
cracked
confidence:  115.56274890899658 %
cracked
confidence:  100.18367767333984 %
cracked
confidence:  87.44404315948486 %
cracked
confidence:  110.2407693862915 %
cracked
confidence:  159.80513095855713 %
cracked
confidence:  145.33603191375732 %
cracked
confidence:  138.52458000183105 %
cracked
confidence:  122.2982406616211 %


cracked
confidence:  145.45083045959473 %
cracked
confidence:  135.4500412940979 %
cracked
confidence:  122.02394008636475 %
cracked
confidence:  118.72187852859497 %
cracked
confidence:  114.26222324371338 %
cracked
confidence:  100.55019855499268 %
cracked
confidence:  89.83484506607056 %
cracked
confidence:  110.76213121414185 %
cracked
confidence:  158.36044549942017 %
cracked
confidence:  146.78349494934082 %
cracked
confidence:  135.87698936462402 %
cracked
confidence:  122.20739126205444 %
cracked
confidence:  118.62236261367798 %
cracked
confidence:  114.88580703735352 %
cracked
confidence:  100.84590911865234 %
cracked
confidence:  89.91443514823914 %
cracked
confidence:  110.84120273590088 %
cracked
confidence:  158.27394723892212 %
cracked
confidence:  147.37870693206787 %
cracked
confidence:  135.5766773223877 %
cracked
confidence:  122.12766408920288 %
cracked
confidence:  118.88870000839233 %
cracked
confidence:  115.00749588012695 %
cracked
confidence:  100.9785175323486

...

cracked
confidence:  152.38046646118164 %
cracked
confidence:  139.35753107070923 %
cracked
confidence:  187.86914348602295 %
cracked
confidence:  150.53435564041138 %
cracked
confidence:  123.13776016235352 %
cracked
confidence:  87.25725412368774 %
cracked
confidence:  106.510329246521 %
cracked
confidence:  148.89084100723267 %
cracked
confidence:  96.61381840705872 %
cracked
confidence:  121.26495838165283 %
cracked
confidence:  190.20916223526 %
cracked
confidence:  99.87007975578308 %
cracked
confidence:  169.9446201324463 %
cracked
confidence:  122.52786159515381 %
cracked
confidence:  111.24613285064697 %
cracked
confidence:  129.47251796722412 %
cracked
confidence:  141.06189012527466 %
cracked
confidence:  174.34561252593994 %
cracked
confidence:  98.17256927490234 %
cracked
confidence:  178.37846279144287 %
cracked
confidence:  94.2958116531372 %
cracked
confidence:  106.03941679000854 %
cracked
confidence:  101.64937973022461 %
cracked
confidence:  108.54324102401733 %
crac

cracked
confidence:  139.73946571350098 %
cracked
confidence:  161.5855574607849 %
cracked
confidence:  173.3159065246582 %
cracked
confidence:  161.311936378479 %
cracked
confidence:  150.3227949142456 %
cracked
confidence:  107.30311870574951 %
cracked
confidence:  91.46894216537476 %
cracked
confidence:  110.36434173583984 %
cracked
confidence:  151.93190574645996 %
cracked
confidence:  182.64391422271729 %
cracked
confidence:  108.61430168151855 %
cracked
confidence:  149.77688789367676 %
cracked
confidence:  166.40267372131348 %
cracked
confidence:  83.12776684761047 %
cracked
confidence:  108.10470581054688 %
cracked
confidence:  106.64409399032593 %
cracked
confidence:  111.32931709289551 %
cracked
confidence:  153.4069538116455 %
cracked
confidence:  183.89480113983154 %
cracked
confidence:  115.50854444503784 %
cracked
confidence:  148.21301698684692 %
cracked
confidence:  168.76355409622192 %
cracked
confidence:  81.7793071269989 %
cracked
confidence:  110.65689325332642 %
cr

cracked
confidence:  96.74373269081116 %
cracked
confidence:  183.69989395141602 %
cracked
confidence:  123.33219051361084 %
cracked
confidence:  119.54488754272461 %
cracked
confidence:  163.12694549560547 %
cracked
confidence:  93.35179328918457 %
cracked
confidence:  182.09068775177002 %
cracked
confidence:  207.1467161178589 %
cracked
confidence:  157.20536708831787 %
cracked
confidence:  97.94545769691467 %
cracked
confidence:  182.94878005981445 %
cracked
confidence:  123.40140342712402 %
cracked
confidence:  120.76263427734375 %
cracked
confidence:  143.1645393371582 %
cracked
confidence:  93.98165941238403 %
cracked
confidence:  181.74140453338623 %
cracked
confidence:  207.0007562637329 %
cracked
confidence:  158.0120086669922 %
cracked
confidence:  98.89543652534485 %
cracked
confidence:  183.12400579452515 %
cracked
confidence:  123.0480432510376 %
cracked
confidence:  121.29013538360596 %
cracked
confidence:  138.80407810211182 %
cracked
confidence:  93.84410381317139 %
cra

cracked
confidence:  120.16149759292603 %
cracked
confidence:  103.52002382278442 %
cracked
confidence:  173.79891872406006 %
cracked
confidence:  129.14845943450928 %
cracked
confidence:  181.59775733947754 %
cracked
confidence:  203.3961534500122 %
cracked
confidence:  171.6480255126953 %
cracked
confidence:  85.96789240837097 %
cracked
confidence:  182.4774980545044 %
cracked
confidence:  118.56452226638794 %
cracked
confidence:  103.98023128509521 %
cracked
confidence:  172.40495681762695 %
cracked
confidence:  131.1805248260498 %
cracked
confidence:  181.50933980941772 %
cracked
confidence:  204.06715869903564 %
cracked
confidence:  171.46600484848022 %
cracked
confidence:  86.29237413406372 %
cracked
confidence:  182.75046348571777 %
cracked
confidence:  119.44622993469238 %
cracked
confidence:  103.77209186553955 %
cracked
confidence:  172.43714332580566 %
cracked
confidence:  132.49343633651733 %
cracked
confidence:  181.37731552124023 %
cracked
confidence:  203.867506980896 %


cracked
confidence:  126.95672512054443 %
cracked
confidence:  103.96459102630615 %
cracked
confidence:  187.49752044677734 %
cracked
confidence:  110.58416366577148 %
cracked
confidence:  180.16990423202515 %
cracked
confidence:  206.1171293258667 %
cracked
confidence:  168.86515617370605 %
cracked
confidence:  91.49887561798096 %
cracked
confidence:  181.88247680664062 %
cracked
confidence:  126.6085147857666 %
cracked
confidence:  103.9555311203003 %
cracked
confidence:  188.84689807891846 %
cracked
confidence:  107.6640248298645 %
cracked
confidence:  180.57482242584229 %
cracked
confidence:  206.03339672088623 %
cracked
confidence:  167.2863483428955 %
cracked
confidence:  91.60146117210388 %
cracked
confidence:  182.1060061454773 %
cracked
confidence:  126.49736404418945 %
cracked
confidence:  105.17630577087402 %
cracked
confidence:  190.25254249572754 %
cracked
confidence:  106.27086162567139 %
cracked
confidence:  180.60436248779297 %
cracked
confidence:  206.62736892700195 %
